In [1]:
import duckdb
import seaborn
import matplotlib
import pygwalker
from load import DATABASE_PATH

seaborn.set_style("darkgrid")

## List all database tables

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql("SHOW ALL TABLES").show()

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    table_details = connection.sql("SHOW ALL TABLES").df()
table_details

## Explore PLUTO

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    pluto_description = connection.sql("DESCRIBE pluto").df()
pluto_description

Database queries are fast

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql("SELECT * FROM pluto").show()

Creating a DataFrame from query results can be slow

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    pluto = connection.sql("SELECT * FROM pluto").df()
pluto

Selecting a subset of columns can be fast

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    pluto_some_columns = connection.sql("SELECT bbl, borough, geom FROM pluto").df()
pluto_some_columns

In [ ]:
pluto_some_columns.value_counts(subset=["borough"])

## Explore Airbnb

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql("DESCRIBE airbnb_nyc_listings").show()
    connection.sql("DESCRIBE airbnb_nyc_detailed_listings").show()

In [10]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    airbnb_nyc_listings = connection.sql("SELECT * FROM airbnb_nyc_listings").df()
    airbnb_nyc_detailed_listings = connection.sql("SELECT * FROM airbnb_nyc_detailed_listings").df()

In [ ]:
airbnb_nyc_listings

In [ ]:
airbnb_nyc_detailed_listings

In [ ]:
print(f"Range of price values: {airbnb_nyc_listings["price"].min()} - {airbnb_nyc_listings["price"].max()}")

In [ ]:
seaborn.displot(airbnb_nyc_listings, x="price", log_scale=True)

In [ ]:
price_histograms = seaborn.displot(
    airbnb_nyc_listings,
    x="price",
    log_scale=True,
    col="neighbourhood_group",
)
for ax in price_histograms.axes.flatten():
    ax.get_xaxis().set_major_formatter(
        matplotlib.ticker.FuncFormatter(
            lambda x, _: format(int(x), ",")
        )
    )
    ax.get_yaxis().set_major_formatter(
        matplotlib.ticker.FuncFormatter(lambda y, _: format(int(y), ","))
    )

`pygwalker` is an interactive way to explore data with a UI similar to Tableau

It can be used with a DataFrame or a database connection

In [ ]:
conn = pygwalker.data_parsers.database_parser.Connector(
    f"duckdb:///{DATABASE_PATH}", "SELECT * FROM airbnb_nyc_listings"
)
pygwalker.walk(conn, kernel_computation=True)

In [ ]:
conn.engine.dispose()

## Explore NYC Taxi and Limousine Commission (TLC) Trip Records

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql("DESCRIBE tlc_trips").show()
    connection.sql("SELECT COUNT(*) FROM tlc_trips").show()

Creating a DataFrame from this table takes several minutes (if it works at all)

A sample is useful to select a random subset of rows

In [ ]:
conn = pygwalker.data_parsers.database_parser.Connector(
    f"duckdb:///{DATABASE_PATH}", "SELECT * FROM tlc_trips USING SAMPLE 5000"
)
pygwalker.walk(conn, kernel_computation=True)

In [ ]:
conn.engine.dispose()

Exploration can inform fast analytical queries

For example,
- How long does the average trip take?
- How long does the average trip take by vehicle type?

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            SELECT
              filename,
              count(*) as number_of_trips
            FROM tlc_trips
            GROUP BY filename
            ORDER BY number_of_trips DESC
        """
    ).show()

Coming from different source files, different vehicle types have different columns for the same information

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            ALTER TABLE tlc_trips DROP COLUMN IF EXISTS vehicle_type;
            ALTER TABLE tlc_trips ADD COLUMN IF NOT EXISTS vehicle_type VARCHAR;
            UPDATE tlc_trips
            SET vehicle_type = (
                CASE
                  WHEN filename LIKE '%yellow%' THEN 'yellow_taxi'
                  WHEN filename LIKE '%green%' THEN 'green_taxi'
                  WHEN filename LIKE '%fhv_trip%' THEN 'fhv'
                  WHEN filename LIKE '%fhvhv%' THEN 'fhvhv'
                END
            )
        """
    )

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            SELECT
              vehicle_type,
              count(*) as number_of_trips
            FROM tlc_trips
            GROUP BY vehicle_type
            ORDER BY number_of_trips DESC
        """
    ).show()

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            ALTER TABLE tlc_trips DROP COLUMN IF EXISTS combined_pickup_datetime;
            ALTER TABLE tlc_trips ADD COLUMN IF NOT EXISTS combined_pickup_datetime timestamp;
            UPDATE tlc_trips
            SET combined_pickup_datetime = (coalesce(lpep_pickup_datetime, tpep_pickup_datetime, Pickup_datetime))
        """
    )
    connection.sql(
        """
            ALTER TABLE tlc_trips DROP COLUMN IF EXISTS combined_dropoff_datetime;
            ALTER TABLE tlc_trips ADD COLUMN IF NOT EXISTS combined_dropoff_datetime timestamp;
            UPDATE tlc_trips
            SET combined_dropoff_datetime = (coalesce(lpep_dropoff_datetime, tpep_dropoff_datetime, DropOff_datetime))
        """
    )

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            ALTER TABLE tlc_trips DROP COLUMN IF EXISTS trip_time;
            ALTER TABLE tlc_trips ADD COLUMN IF NOT EXISTS trip_time interval;
            UPDATE tlc_trips
            SET trip_time = (combined_dropoff_datetime - combined_pickup_datetime)
        """
    )

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            SELECT
              combined_pickup_datetime,
              combined_dropoff_datetime,
              trip_time
            FROM tlc_trips
        """
    ).show()

In [ ]:
with duckdb.connect(str(DATABASE_PATH)) as connection:
    connection.sql(
        """
            SELECT
                vehicle_type,
                count(*) as number_of_trips,
                round(avg(extract('minute' FROM trip_time)), 1) as average_trip_time_mintues
            FROM tlc_trips
            GROUP BY vehicle_type
            ORDER by average_trip_time_mintues ASC
        """
    ).show()